<a href="https://colab.research.google.com/github/ariG23498/GrokkingDeepLearning/blob/master/%2310_SentimentIMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import pandas as pd
import nltk
import string
from sklearn.utils import shuffle
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/My Drive/Datasets/imdb_master.csv', encoding='latin-1')
df = df.drop(columns=['Unnamed: 0'])
df = shuffle(df)
df.head()

,type,review,label,file
48969,train,My wife and I took our 13 year old son to see ...,pos,9073_10.txt
46544,train,"""Once upon a time there was a charming land ca...",pos,6891_9.txt
8279,test,I didn't know what to expect when I started wa...,neg,6201_1.txt
51621,train,"We've seen it many times before. From ""Miami V...",unsup,1145_0.txt
20203,test,"So well made, no CGI crap. Has anyone else bee...",pos,5684_8.txt


In [4]:
df['label'].unique()

array(['pos', 'neg', 'unsup'], dtype=object)

In [5]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

df['label'].unique()

array([1, 0, 2])

In [6]:
df.head()

,type,review,label,file
48969,train,My wife and I took our 13 year old son to see ...,1,9073_10.txt
46544,train,"""Once upon a time there was a charming land ca...",1,6891_9.txt
8279,test,I didn't know what to expect when I started wa...,0,6201_1.txt
51621,train,"We've seen it many times before. From ""Miami V...",2,1145_0.txt
20203,test,"So well made, no CGI crap. Has anyone else bee...",1,5684_8.txt


In [0]:
train = df[df['type'] == 'train'].drop(columns=['type','file']).values
test = df[df['type'] == 'test'].drop(columns=['type','file']).values

In [8]:
print('Length of Train: {}'.format(len(train)))
print('Length of Test: {}'.format(len(test)))

Length of Train: 75000
Length of Test: 25000


# Pre-processing

In [0]:
def preprocess(sentence):
  # Tokenize Contents
  contentsTokenized = nltk.tokenize.word_tokenize(sentence)

  # Remove the stop_words
  stop_word_set = set(nltk.corpus.stopwords.words("english"))
  filteredContents_afterstop = []
  for word in contentsTokenized:
    if word not in stop_word_set:
      filteredContents_afterstop.append(word)
  
  # Performing porterStemming
  porterStemmer = nltk.stem.PorterStemmer()
  filteredContents = [porterStemmer.stem(word) for word in filteredContents_afterstop]

  # Remove Punctuations
  excludePunctuation = set(string.punctuation)
    
  # manually add additional punctuation to remove
  doubleSingleQuote = '\'\''
  doubleDash = '--'
  doubleTick = '``'

  excludePunctuation.add(doubleSingleQuote)
  excludePunctuation.add(doubleDash)
  excludePunctuation.add(doubleTick)

  filteredContents_afterpunc = []
  for word in filteredContents_afterstop:
    if word not in excludePunctuation:
      filteredContents_afterpunc.append(word)

  # Convert all to lower case
  filteredContents_lower = [term.lower() for term in filteredContents_afterpunc]
  return filteredContents_lower

In [0]:
# token is a list of set
# each set consists of words in the particular review
token_train = list(map(lambda x: set(preprocess(x)),train[:,0]))
token_test = list(map(lambda x: set(preprocess(x)),test[:,0]))

In [11]:
print(len(token_train))
print(len(token_test))

75000
25000


In [12]:
# Total vocabulary
vocab = set()
for sentence in token_train:
    for word in sentence:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

print('Total vocabulary count TRAIN: {}'.format(len(vocab)))

Total vocabulary count TRAIN: 217750


In [13]:
# For each word in vocab there is an index TRAIN
word2index = {}
for i,word in enumerate(vocab):
    word2index[word] = i
print('Total word2index count: {}'.format(len(word2index)))

Total word2index count: 217750


In [0]:
# input_dataset is used to keep the indices of each words in each sentence TRAIN
input_dataset_train = list()
for sentence in token_train:
    sent_indices = list()
    for word in sentence:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset_train.append(list(sent_indices))

# input_dataset is used to keep the indices of each words in each sentence TEST
input_dataset_test = list()
for sentence in token_test:
    sent_indices = list()
    for word in sentence:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset_test.append(list(sent_indices))

In [0]:
X_train = input_dataset_train
y_train = train[:,1]

X_test = input_dataset_test
y_test = test[:,1]

In [0]:
import numpy as np
np.random.seed(1)

In [0]:
one_hot_train = np.zeros((len(y_train),3))
for i,l in enumerate(y_train):
  one_hot_train[i][l] = 1

one_hot_test = np.zeros((len(y_test),3))
for i,l in enumerate(y_test):
  one_hot_test[i][l] = 1

In [0]:
y_train = one_hot_train
y_test = one_hot_test

In [19]:
print(X_train[0][:10])
print(y_train[0])
print(X_test[0][:10])
print(y_test[0])

[122278, 30004, 26102, 67433, 155704, 98065, 17091, 197385, 67860, 51747]
[0. 1. 0.]
[208108, 207395, 75374, 110641, 195337, 124033, 96217, 195619, 8702, 126591]
[1. 0. 0.]


In [0]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

In [0]:
alpha, iterations = (0.01, 30)
hidden_size = 1000

weights_0_1 = 0.2 * np.random.random((len(vocab),hidden_size))
weights_1_2 = 0.2 * np.random.random((hidden_size,3)) - 0.1

In [26]:
correct, total = (0,0)
for iter in range(iterations):
  # Enters each epoch
  for i in range(len(X_train)):
    # For individual review
    x,y = (X_train[i],y_train[i])

    # FORWARD PROP
    layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
    # layer_1 --> (100,)
    # layer_2 --> (3,)

    # BACKWARD PROP
    layer_2_delta = layer_2 - y
    layer_1_delta = layer_2_delta.dot(weights_1_2.T)

    weights_0_1[x] -= layer_1_delta * alpha
    weights_1_2 -= np.outer(layer_1,layer_2_delta) * alpha
    if(np.argmax(layer_2) == np.argmax(y)):
      correct +=1
    total += 1
print('TRAIN Acc:', correct/total)

TRAIN Acc: 0.9480164444444444


In [27]:
correct,total = (0,0)
for i in range(len(X_test)):
  x = X_test[i]
  y = y_test[i]

  layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
  layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
  
  if(np.argmax(layer_2) == np.argmax(y)):
    correct +=1
  total += 1
print('TEST Acc:', correct/total)

TEST Acc: 0.22496
